## Animate diff

In [ ]:
import path
path.setPathLocalNotebook()

from diffuserslib import DiffusersPipelines, DiffusersBaseModel, GenerationParameters, ModelParameters

pipelines = DiffusersPipelines(device = "mps", safety_checker=True, cache_dir=None)
pipelines.loadPresetFile("../../modelconfig.yml")

params = GenerationParameters(
    generationtype = "animatediff",
    prompt = "a photo of an astronaut riding a horse on mars", 
    negprompt = "", 
    steps = 40, 
    cfgscale = 7, 
    width = 512, 
    height = 512, 
    frames = 16,
    scheduler = "EulerDiscreteScheduler", 
    seed = None, 
    models = [ModelParameters(name = "runwayml/stable-diffusion-v1-5")])
frames, seed = pipelines.generate(params)

print(f"Seed: {seed}")

# for image in images:
for frame in frames:
    display(frame)

## SD video

In [ ]:
import path
path.setPathLocalNotebook()

from diffuserslib import DiffusersPipelines, ControlImageParameters, GenerationParameters, ModelParameters
from PIL import Image

pipelines = DiffusersPipelines(device = "mps", safety_checker=True, cache_dir=None)
pipelines.loadPresetFile("../../modelconfig.yml")

image = Image.open("data/rabbit.png")

params = GenerationParameters(
    generationtype = "generate",
    seed = None, 
    controlimages = [ControlImageParameters(image = image)],
    models = [ModelParameters(name = "stabilityai/stable-video-diffusion-img2vid")])
frames, seed = pipelines.generate(params)

print(f"Seed: {seed}")

# for image in images:
for frame in frames:
    display(frame)

## Video pipelines

In [ ]:
import path
path.setPathLocalNotebook()

from PIL import Image
from diffuserslib import (ImageProcessorPipeline, FrameSequenceProcessor, InitImageProcessor, ResizeProcessor,
                          PoseDetectionProcessor, RotateMovementProcessor, PlaceholderArgument, linearInterpolation,
                          VideoProcessorPipeline, DiffusionGeneratorProcessor, ControlImageType)

pipelines = DiffusersPipelines(device = "mps", safety_checker=False)
pipelines.loadPresetFile("../../modelconfig.yml")

initProcessor = ImageProcessorPipeline()
initProcessor.addTask(InitImageProcessor(image=Image.open("data/pose.png")))
initProcessor.addTask(ResizeProcessor(size=(512, 512)))

frameProcessor = ImageProcessorPipeline()
frameProcessor.addTask(InitImageProcessor(image=PlaceholderArgument("feedforward")))
frameProcessor.addTask(RotateMovementProcessor(angle=90, interpolation=linearInterpolation))
frameProcessor.addTask(PoseDetectionProcessor())
frameProcessor.addTask(DiffusionGeneratorProcessor(pipelines = pipelines, prompt = "girl", model = "runwayml/stable-diffusion-v1-5", 
                                                   controlimages = [ ControlImageParameters(type = ControlImageType.IMAGETYPE_CONTROLIMAGE, model = "lllyasviel/control_v11p_sd15_openpose") ]
                                                   ))


renderer = FrameSequenceProcessor(initImage=initProcessor, frameProcessor=frameProcessor, frames=4, feedForwardIndex=1)
frames = renderer()
